# A notebook for creating sc graph animations with glimpses

In [ ]:
import napari
from cellpose import models
from octopuslite import utils, tile
import numpy as np
import os

def view(img):
    return napari.Viewer().add_image(img)
from napari_animation import Animation
from tqdm.auto import tqdm

import btrack
import dask.array as da

from pathlib import Path

from skimage.transform import rescale, resize, downscale_local_mean
from skimage.io import imsave, imread
from scipy.ndimage.filters import gaussian_filter1d

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")
sns.set_palette("rocket_r")

def msd_calc(x1, y1, x2, y2):
    """
    Displacement calculation for cell movement between frames
    """
    return np.sqrt((x1-x2)**2+(y1-y2)**2)

def update_slider(event):
    # only trigger if update comes from first axis (optional)
        #ind_lambda = viewer.dims.indices[0]
    time = viewer.dims.current_step[0]
    viewer.text_overlay.text = f"{time:1.1f} hours"
text_size = 18
napari_scale = [1.4949402023919043e-07, 1.4949402023919043e-07]

import glob
from natsort import natsorted

### Load all metadata

Both the image metadata and the assay layout metadata

In [ ]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
image_metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(image_metadata_fn)
assay_layout_metadata = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = utils.read_harmony_metadata(assay_layout_metadata, assay_layout=True)
assay_layout

# Iteratively load all tracks

and append to a track_dict dictionary

In [ ]:
tracks_dict = dict()
### iterate over all experimental conditions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):
    ### load tracks
    with btrack.dataio.HDF5FileHandler(
            f"/mnt/DATA/macrohet/segmentation/tracks_objs/({row},{column})_tracks_rescaled.h5", 
            'r', 
            obj_type = 'obj_type_1', 
            ) as hdf: 
            tracks = hdf.tracks
            objs = hdf.objects
    ### append tracks to dictionary
    tracks_dict[(row, column)] = tracks

# Compile all full length tracks into dataframe

Add extra information such as the MSD of cells between frames

In [ ]:
### list of track info dfs
dfs = list()
### empty dictionary for filtered tracks
filtered_tracks = dict()
### iterate over all tracks
for key in tracks_dict.keys():
    ### extract tracks only with max length
    filtered_tracks[key] = [track for track in tracks_dict[key] if len(track) == 75]
    ### iterate over full length tracks
    for track in filtered_tracks[key]:
        ### get info for assay layout
        info = assay_layout.loc[key]
        ### compile single track dictionary of info
        d = {'Time (hours)':track['t'], 
             'x':track['x'],
             'y':track['y'],
             'Area':track['area'], 
             'Intracellular Mtb content':track['mean_intensity-1'],
             'Mean Mtb content':[np.nanmean(track['mean_intensity-1']) for i in range(len(track['t']))],
             'Macroph. GFP expression':track['mean_intensity-0'],
             'Eccentricity':np.sqrt(1-((track['minor_axis_length']**2)/(track['major_axis_length']**2))),
             'MSD': [msd_calc(track['x'][i-1], 
                              track['y'][i-1], 
                              track['x'][i], 
                              track['y'][i]) 
                              if i != 0 else 0
                              for i in range(0, len(track))],
             'Strain':[info['Strain'] for i in range(len(track['t']))], 
             'Compound':[info['Compound'] for i in range(len(track['t']))], 
             'Concentration':[info['ConcentrationEC'] for i in range(len(track['t']))], 
             'Cell ID':[track.ID for i in range(len(track['t']))],
             'Acquisition ID':[key for i in range(len(track['t']))]}
        ### append df to list of dfs
        dfs.append(pd.DataFrame(d))
### concat single track dfs into big df
df = pd.concat(dfs, ignore_index=True)
### interpolate missing values as sometimes segmentation drops result in NaN
df.interpolate(inplace=True)

In [ ]:
df

### Add category to discern initial amount of Mtb growth

Categorical

In [ ]:
initial_mtb = df.loc[df['Time (hours)'] == 0, 'Intracellular Mtb content']
initial_mtb_quartiles = pd.cut(initial_mtb, bins = [initial_mtb.quantile(.0), 
                                                    initial_mtb.quantile(.25), 
                                                    initial_mtb.quantile(0.5), 
                                                    initial_mtb.quantile(.75),
                                                    initial_mtb.quantile(1)], 
                                                    labels = ['Lower', 'Lower-mid', 'Upper-mid', 'Upper'])
df['Initial Mtb load (quartile)'] = initial_mtb_quartiles
df.fillna(method='ffill', inplace=True)

In [ ]:
final_mtb = df.loc[df['Time (hours)'] == 74, 'Intracellular Mtb content']
final_mtb_quartiles = pd.cut(final_mtb, bins = [final_mtb.quantile(.0), 
                                                    final_mtb.quantile(.25), 
                                                    final_mtb.quantile(0.5), 
                                                    final_mtb.quantile(.75),
                                                    final_mtb.quantile(1)], 
                                                    labels = ['Lower', 'Lower-mid', 'Upper-mid', 'Upper'])
df['Final Mtb load (quartile)'] = final_mtb_quartiles
df.fillna(method='bfill', inplace=True)

Now continuous assessment: ie. the actual value of the Mtb content in the first frame

In [ ]:
initial_mtb = df.loc[df['Time (hours)'] == 0, 'Intracellular Mtb content']
df['Initial Mtb load'] = initial_mtb
df.fillna(method='ffill', inplace=True)

In [ ]:
final_mtb = df.loc[df['Time (hours)'] == 74, 'Intracellular Mtb content']
df['Final Mtb load'] = final_mtb
df.fillna(method='bfill', inplace=True)

In [ ]:
sns.relplot(data = df[df['Compound'] == 'PZA'], 
            x = 'Time (hours)', 
            y = 'Intracellular Mtb content', 
            kind = 'line',
            col = 'Concentration',
            hue = 'Concentration', 
            aspect = 0.75,
           )

In [ ]:
selected_expt_df = df[(df['Compound'] == 'PZA') & (df['Concentration'] == 'EC50')]

In [ ]:
extreme_cases = pd.concat([selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.max(selected_expt_df['Mean Mtb content']))],selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.min(selected_expt_df['Mean Mtb content']))]])

In [ ]:
sns.lineplot(data = extreme_cases, 
            x = 'Time (hours)', 
            y = 'Intracellular Mtb content', 
            hue = 'Cell ID', 
           )
sns.despine()

## Pick a single experiment to focus on?

In [ ]:
extreme_cases

#### pull corresponding images

In [ ]:
row, column = extreme_cases['Acquisition ID'].iloc[0]
images = tile.compile_mosaic(
                             image_dir, 
                             metadata, 
                             row, 
                             column, 
                             set_plane = 'sum_proj',
                             ).astype(np.uint16)

In [ ]:
images

## Select one cell as a seperate df

In [ ]:
cell_IDs = list(set(extreme_cases['Cell ID']))

In [ ]:
sc_df = extreme_cases[extreme_cases['Cell ID']==cell_IDs[0]]
ID = list(extreme_cases['Cell ID'])[0]
acq_ID = list(extreme_cases['Acquisition ID'])[0]

## Make series of glimpses

In [ ]:
### glimpse size
size = 500
### resized images
scale = 6048/1200

In [ ]:
%%time
glimpse_stack = list()
for row in tqdm(sc_df.iterrows(), total = len(sc_df)):
    time, x, y = row[1]['Time (hours)'], row[1]['y'], row[1]['x']
    frame = images[time,...]
    x1, y1 = x*scale, y*scale
#     x1, x2, y1, y2 = x1*scale, x2*scale, y1*scale, y2*scale
#     x1, x2, y1, y2 = x1, x2+size, y1, y2+size
    x1, x2, y1, y2 = x1, x1+size, y1, y1+size
    frame = da.pad(frame, [(0, 0), (size/2, size/2), (size/2, size/2)], 'constant', constant_values = 0) 
    glimpse = frame[..., int(x1): int(x2), int(y1): int(y2)]# frame[..., int(x1): int(x2), int(y1): int(y2)]

    glimpse_stack.append(glimpse)
glimpse_stack = np.stack(glimpse_stack, axis = 1)

In [ ]:
%%time
glimpse_stack = glimpse_stack.compute().compute()

In [ ]:
glimpse_stack.shape

### Check glimpse with labels

In [ ]:
viewer = napari.Viewer()
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
                )
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

In [ ]:
zoom = viewer.camera.zoom
cam_coords = viewer.camera.center

### Use camera angle for animation/mp4 creation

In [ ]:
fn = f'/home/dayn/Videos/tb_mp4s/glimpses/pierre_data/{acq_ID}/{ID}/glimpse_{ID}.mp4'
Path(os.path.dirname(fn)).mkdir(parents=True, exist_ok=True)

viewer = napari.Viewer()
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
                )
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

animation = Animation(viewer)
viewer.update_console({'animation': animation})
# viewer.camera.center = (0, 0, 3024, 3024)
viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
viewer.camera.zoom = zoom*0.85
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (74.0,  cam_coords[-2], cam_coords[-1])
animation.capture_keyframe(steps = 100)

animation.animate(fn, 
                  canvas_only=True,
                  fps = 5,
                  quality = 9)
viewer.close()

Path(os.path.dirname(os.path.dirname(fn)+f'/{ID}_glimpse_seq')).mkdir(parents=True, exist_ok=True)

viewer = napari.Viewer()
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
                )
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

for t in tqdm(list(sc_df['Time (hours)'])):
    viewer.dims.current_step = (t, cam_coords[-2], cam_coords[-1])
    viewer.camera.zoom = zoom*0.85
    new_fn = os.path.dirname(fn)+f'/{ID}_glimpse_seq/t_{t}.tiff'
    imsave(new_fn, viewer.screenshot())

# Plot animated graph

In [ ]:
y.min()

In [ ]:
Path(os.path.dirname(new_fn).replace('_seq', f'_animated_graph/')).mkdir(parents = True, exist_ok = True)

y = list(sc_df['Intracellular Mtb content'].interpolate())
y = gaussian_filter1d(y, sigma=1.5)
for n, row in tqdm(enumerate(sc_df.iterrows()), total = len(sc_df)):
#     x = list(cell_ID_431['Time'].iloc[0:n+1])
#     y = list(cell_ID_431['Intracellular Mtb content'].iloc[0:n+1].interpolate())
    x = list(sc_df['Time (hours)'].iloc[0:n+1])
    
    plot_y = y[0:n+1]

    t = row[1]['Time (hours)'] 
    ID = row[1]['Cell ID']
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,8), gridspec_kw={'width_ratios': [1.15, 1]})
#     fig.suptitle(f'Cell ID: {ID}')
    glimpse_fn = os.path.dirname(new_fn)+f'/t_{t}.tiff'
    glimpse_img = imread(glimpse_fn)
    ax1.imshow(glimpse_img)

    ax1.axis('off')
    ax2.plot(x, plot_y,)# c = palette[1])
    ax2.set_aspect(75/y.max())
    ax2.set(xlabel = 'Time (hours)', ylabel = f'Mtb expression (rfp intensity) in cell {ID}', ylim=(y.min()*0.85,y.max()*1.15), xlim = (0,75))
    sns.despine()
    new_graph_fn = os.path.dirname(new_fn).replace('_seq', f'_animated_graph/smooth_t_{t}.png')
    plt.savefig(new_graph_fn, bbox_inches = 'tight', dpi = 314)

## Compile into mp4

In [ ]:
plots = list()
for fn in tqdm(natsorted(glob.glob(os.path.dirname(new_graph_fn)+'/smooth_t_*.png')), ):#total = 56):
    plot = imread(fn)
    plots.append(plot)
plots = np.stack(plots, axis = 0)

In [ ]:
final_fn = os.path.dirname(new_graph_fn)+'.mp4'

In [ ]:
viewer = napari.Viewer()
viewer.add_image(plots)#, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
            #    )
viewer.theme = 'light'
# viewer.scale_bar.visible = True
# viewer.scale_bar.unit = 'm'
# viewer.scale_bar.font_size = text_size
# viewer.text_overlay.visible = True
# viewer.text_overlay.color = 'black'
# viewer.text_overlay.position = 'bottom_left'
# viewer.text_overlay.font_size = text_size
# viewer.dims.events.current_step.connect(update_slider)

animation = Animation(viewer)
viewer.update_console({'animation': animation})
# # viewer.camera.center = (0, 0, 3024, 3024)
viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
viewer.camera.zoom = zoom
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (75.0,  cam_coords[-2], cam_coords[-1])
animation.capture_keyframe(steps = 100)

animation.animate(final_fn, 
                  canvas_only=True,
                  fps = 5,
                  quality = 9)
viewer.close()

# Iteratively generate glimpses

In [ ]:
### glimpse size
size = 500
### resized images
scale = 6048/1200

In [ ]:
selected_expt_df = df[(df['Compound'] == 'PZA') & (df['Concentration'] == 'EC50')]
extreme_cases = pd.concat([selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.max(selected_expt_df['Mean Mtb content']))],
                           selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.min(selected_expt_df['Mean Mtb content']))]])

In [ ]:
for assay_info in tqdm(assay_layout.iterrows(), 
                       total = len(assay_layout), 
                       desc = 'Iterating over different assays, generating two extreme glimpses per assay'):
    ### pull acq ID from assay info
    acq_ID = assay_info[0]

    ### iterate over different experiments
    selected_expt_df = df[(df['Acquisition ID'] == acq_ID)]

    ### get extreme cells
    extreme_cases = pd.concat([selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.max(selected_expt_df['Mean Mtb content']))],
                               selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.min(selected_expt_df['Mean Mtb content']))]])
    
    ### iterate over two extreme cases in either
    for ID in tqdm(extreme_cases['Cell ID'].unique(), 
                   desc = f'Iterating over two extreme cells in acquisition ID:{acq_ID}'):
        ### isolate a single cell ID from df of interesting cells
        sc_df = extreme_cases[extreme_cases['Cell ID']==ID]
        # ### get cell ID
        # ID = sc_df['Cell ID'].iloc[0]
        # ### get acquisition ID 
        # acq_ID = sc_df['Acquisition ID'].iloc[0]
        ### use acq ID to pre load correct images
        row, column = acq_ID
        images = tile.compile_mosaic(
                                     image_dir, 
                                     metadata, 
                                     row, 
                                     column, 
                                     set_plane = 'sum_proj',
                                     ).astype(np.uint16)
        ### create base dirname
        basedir = f'/home/dayn/Videos/tb_mp4s/glimpses/pierre_data/{acq_ID}/{ID}/'
        Path(os.path.dirname(basedir)).mkdir(parents=True, exist_ok=True)
        ### make glimpse stack of images
        print(f'Creating glimpse ID: {acq_ID, ID}')
        ### create empty list for stack of images
        glimpse_stack = list()
        ### iterate over time points from single cell 
        for row in tqdm(sc_df.iterrows(), total = len(sc_df), desc = f'Creating glimpse ID: {acq_ID, ID}'):
            ### get coords
            t, x, y = row[1]['Time (hours)'], row[1]['y'], row[1]['x']
            ### select proper frame
            frame = images[t,...]
            ### scale as tracking was done on rescaled images
            x1, y1 = x*scale, y*scale
            ### create window for glimpse
            x1, x2, y1, y2 = x1, x1+size, y1, y1+size
            ### add padding for boundary cases
            frame = da.pad(frame, [(0, 0), (size/2, size/2), (size/2, size/2)], 'constant', constant_values = 0) 
            ### create glimpse image by cropping original image
            glimpse = frame[..., int(x1): int(x2), int(y1): int(y2)]
            ### append to glimpse stack
            glimpse_stack.append(glimpse)
        ### stack glimpse together
        glimpse_stack = np.stack(glimpse_stack, axis = 1)
        ### load glimpse into memory
        print(f'Loading glimpse stack {acq_ID, ID} into memory (can take several minutes)')
        glimpse_stack = glimpse_stack.compute().compute()
        ### Check glimpse with labels
        viewer = napari.Viewer()
        viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = napari_scale)
        viewer.theme = 'light'
        viewer.scale_bar.visible = True
        viewer.scale_bar.unit = 'm'
        viewer.scale_bar.font_size = text_size
        viewer.text_overlay.visible = True
        viewer.text_overlay.color = 'black'
        viewer.text_overlay.position = 'bottom_left'
        viewer.text_overlay.font_size = text_size
        viewer.dims.events.current_step.connect(update_slider)
        zoom = viewer.camera.zoom
        cam_coords = viewer.camera.center

        ### create glimpse .mp4 filename
        mp4_fn = os.path.join(basedir, f'glimpse_{ID}.mp4')
        # Path(os.path.dirname(mp4_fn)).mkdir(parents=True, exist_ok=True)
        ### launch napari and animate images into mp4
        viewer = napari.Viewer()
        viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = napari_scale)
        viewer.theme = 'light'
        viewer.scale_bar.visible = True
        viewer.scale_bar.unit = 'm'
        viewer.scale_bar.font_size = text_size
        viewer.text_overlay.visible = True
        viewer.text_overlay.color = 'black'
        viewer.text_overlay.position = 'bottom_left'
        viewer.text_overlay.font_size = text_size
        viewer.dims.events.current_step.connect(update_slider)
        ### initiate animation viewer
        animation = Animation(viewer)
        viewer.update_console({'animation': animation})
        # viewer.camera.center = (0, 0, 3024, 3024)
        viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
        viewer.camera.zoom = zoom*0.85
        animation.capture_keyframe(steps = 100)
        viewer.dims.current_step = (74.0,  cam_coords[-2], cam_coords[-1])
        animation.capture_keyframe(steps = 100)
        ### Save glimpse as MP4
        print(f'Saving glimpse .mp4 ID: {acq_ID, ID}')
        animation.animate(mp4_fn, 
                          canvas_only=True,
                          fps = 5,
                          quality = 9)
        viewer.close()
        ### save glimpse as a series of images
        print(f'Creating glimpse image sequence ID: {acq_ID, ID}')
        ### create output directory
        glimpse_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_seq')
        Path(glimpse_seq_basedir).mkdir(parents=True, exist_ok=True)
        ### create napari instances and use to save glimpse frames with scale bar and time
        viewer = napari.Viewer()
        viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = napari_scale)
        viewer.theme = 'light'
        viewer.scale_bar.visible = True
        viewer.scale_bar.unit = 'm'
        viewer.scale_bar.font_size = text_size
        viewer.text_overlay.visible = True
        viewer.text_overlay.color = 'black'
        viewer.text_overlay.position = 'bottom_left'
        viewer.text_overlay.font_size = text_size
        viewer.dims.events.current_step.connect(update_slider)
        ### save image sequence
        for t in tqdm(list(sc_df['Time (hours)']), desc = f'Saving glimpse image sequence ID: {acq_ID, ID}'):
            viewer.dims.current_step = (t, cam_coords[-2], cam_coords[-1])
            viewer.camera.zoom = zoom*0.85
            glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
            imsave(glimpse_seq_fn, viewer.screenshot())
        viewer.close()
        ### create sequence of graph images
        print(f'Creating graph image sequence ID: {acq_ID, ID}')
        # create graph output folder
        glimpse_graph_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_graph_seq')
        Path(glimpse_graph_seq_basedir).mkdir(parents = True, exist_ok = True)
        ### create graph by interpolating missing y values
        y = sc_df['Intracellular Mtb content'].interpolate().values
        x = sc_df['Time (hours)'].values
        ### smooth graph
        y = gaussian_filter1d(y, sigma=1.5)
        ### iterate over time points
        for n, row in tqdm(enumerate(sc_df.iterrows()), total = len(sc_df), 
                           desc = f'Saving graph image sequence ID: {acq_ID, ID}'):
            ### select portion of data to plot up to current time point iteration
            plot_x = sc_df['Time (hours)'].iloc[0:n+1].values
            plot_y = y[0:n+1]
            ### get time ID for labelling purposes
            t = row[1]['Time (hours)']     
            ### initiate plot with two components
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5), gridspec_kw={'width_ratios': [1, 1]})
            ### label fn with time point
            glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
            ### load glimpse image from this timepoint
            glimpse_img = imread(glimpse_seq_fn)
            ### add glimpse frame to first part of subplot
            ax1.imshow(glimpse_img)
            ### turn axis off for glimpse
            ax1.axis('off')
            ### plot portion of graph 
            ax2.plot(plot_x, plot_y,)# c = palette[1])
            ### set aspect
            # ax2.set_aspect(75/y.max())
            ### label graph properly 
            ax2.set(xlabel = 'Time (hours)', 
                    ylabel = f'Mtb content of cell ID: {ID} \n (raw RFP intensity)', 
                    ylim=(y.min()*0.85,y.max()*1.15), 
                    xlim = (0,75))
            ### despine plot
            sns.despine()
            ### new fn for joint glimpse plot
            new_graph_fn = os.path.join(glimpse_graph_seq_basedir, f'glimpse_graph_{ID}_t_{t}.tiff')
            plt.savefig(new_graph_fn, bbox_inches = 'tight', dpi = 314)

        ## Compile single frame graphs into stack for animation
        print(f'Creating graph and glimpse image sequence ID: {acq_ID, ID}')
        plots = list()
        for fn in natsorted(glob.glob(os.path.join(glimpse_graph_seq_basedir, '*.tiff'))):
            plot = imread(fn)
            plots.append(plot)
        plots = np.stack(plots, axis = 0)
        ### create final fn
        glimpse_graph_fn = os.path.join(basedir, f'glimpse_graph_{ID}.mp4')
        ### initiate napari session for animation
        viewer = napari.Viewer()
        viewer.add_image(plots)#, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
        viewer.theme = 'light'
        # viewer.scale_bar.visible = True
        # viewer.scale_bar.unit = 'm'
        # viewer.scale_bar.font_size = text_size
        # viewer.text_overlay.visible = True
        # viewer.text_overlay.color = 'black'
        # viewer.text_overlay.position = 'bottom_left'
        # viewer.text_overlay.font_size = text_size
        # viewer.dims.events.current_step.connect(update_slider)
        animation = Animation(viewer)
        viewer.update_console({'animation': animation})
        # # viewer.camera.center = (0, 0, 3024, 3024)
        viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
        viewer.camera.zoom = 0.23
        animation.capture_keyframe(steps = 100)
        viewer.dims.current_step = (75.0,  cam_coords[-2], cam_coords[-1])
        animation.capture_keyframe(steps = 100)
        print(f'Saving graph and glimpse animation ID: {acq_ID, ID}')
        animation.animate(glimpse_graph_fn, 
                          canvas_only=True,
                          fps = 5,
                          quality = 9)
        viewer.close()
        plt.clf()